In [1]:
from everything import *  # https://github.com/fnielsen/everything

In [2]:
from dasem.dannet import Dannet

In [3]:
dannet = Dannet()

In [4]:
# Query for sentiment words in DanNet
query = """
select label, value
from (select sa.synset_id, sa.value
      from synset_attributes sa
      where sa.type = 'connotation'), synsets as s
where s.id = synset_id
"""
df = dannet.db.query(query)
df.head()

Indexing schema. This will take a second...finished!


,label,value
0,{børnehave_1_3},negative
1,{mafia_1_2},negative
2,{sammenrend_1; sammensurium_1},negative
3,{tingeltangel_1},negative
4,{kongerække_1_1},positive


In [5]:
# Extract words from the label
dannet_words = {}
for idx, row in df.iterrows():
    synset_words = row['label'].split(';')
    for word in synset_words:
        dannet_words[word[1:].split('_')[0]] = row['value']

In [6]:
# Compare with development version of AFINN
try:
    missing_words, swapped_words = [], []
    filename = expanduser('~/fnielsen/data/Nielsen2011Sentiment_afinndk.txt')
    afinn = read_csv(filename, sep='\t', encoding='utf-8', header=None)
    for dannet_word, dannet_valence in dannet_words.iteritems():
        if dannet_word not in afinn[0].values:
            missing_words.append((dannet_word, dannet_valence))
        else:
            afinn_valence = afinn.ix[afinn[0]==dannet_word, 1].values[0]
            if ((dannet_valence == 'positive' and afinn_valence < 0) or 
                (dannet_valence == 'negative' and afinn_valence > 0)):
                swapped_words.append((dannet_word, dannet_valence))
    pprint(sorted(swapped_words))
    pprint(sorted(missing_words))
except:
    pass

[(u'dr\xf8mme', u'negative')]
[(u'agterlanterne', u'negative'),
 (u'amazone', u'positive'),
 (u'andenklassesborger', u'negative'),
 (u'andenrangsborger', u'negative'),
 (u'angloman', u'negative'),
 (u'antikvitet', u'negative'),
 (u'arbejdsnarkoman', u'negative'),
 (u'artist', u'positive'),
 (u'auteur', u'positive'),
 (u'avisskriveri', u'negative'),
 (u'baby', u'positive'),
 (u'badenymfe', u'positive'),
 (u'bamse', u'yes'),
 (u'bamsefar', u'yes'),
 (u'bandit', u'negative'),
 (u'barnerumpe', u'negative'),
 (u'barner\xf8v', u'negative'),
 (u'baryler', u'negative'),
 (u'bedsteborger', u'negative'),
 (u'behagelighed', u'positive'),
 (u'bengel', u'negative'),
 (u'ber\xf8mthed', u'positive'),
 (u'besk', u'negative'),
 (u'betonbyggeri', u'negative'),
 (u'betonkasse', u'negative'),
 (u'betonklods', u'negative'),
 (u'betonsilo', u'negative'),
 (u'biks,1', u'negative'),
 (u'billig', u'negative'),
 (u'bisse,1', u'negative'),
 (u'blegn\xe6b', u'negative'),
 (u'blodsuger', u'negative'),
 (u'blomster